# 1. Install and Import Dependencies

In [ ]:
!pip install torch transformers requests beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.7 MB/s 
     |████████████████████████████████| 7.6 MB 58.5 MB/s 
     |████████████████████████████████| 182 kB 73.3 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import torch
import requests
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bs4 import BeautifulSoup

# 2. Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3. Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode('This meal was terrible. I would not dine here again.', return_tensors='pt')

In [ ]:
tokenizer.decode(tokens[0])

'[CLS] this meal was terrible. i would not dine here again. [SEP]'

In [ ]:
result = model(tokens)

In [ ]:
result.logits

tensor([[ 4.2673,  2.2746, -0.2436, -2.8600, -2.7331]],
       grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits)) + 1 #Our sentiment rating

1

# 4. Collect Reviews

In [ ]:
r = requests.get('https://www.yelp.com/biz/square-pie-guys-san-francisco')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

In [ ]:
reviews[1]

"We've ordered several of these to the office and they are always a hit with the staff. The turnaround time on the website is very accurate and if you're in a rush, get your order in before the lunch crowd. The only downside here is the lack of a half sized option (in case you're working from home and want to avoid having a pizza this filling for lunch and for dinner)."

# 5. Load Reviews Into DataFrame. Score.

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df.head()

,review
0,I love that this is walking distance from my b...
1,We've ordered several of these to the office a...
2,Ordered delivery from here last night with my ...
3,Ordered some takeout the other night and had n...
4,We arrived in town and got to see our son. Dur...


In [ ]:
df.tail()

,review
6,Review is for the Melissa King (Top Chef fame)...
7,Was excited to try the Souvla x SPG pizza and ...
8,The food here is good and creative. They have ...
9,Pizza was tasty. Pretty simple to get it deliv...
10,"Thanks to Yelp, I found this little pizza plac..."


In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1

In [ ]:
#Test on a review
sentiment_score(df['review'].iloc[1])

4

In [ ]:
#Calculate sentiment for each review. Limit to 512 tokens.
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,review,sentiment
0,I love that this is walking distance from my b...,5
1,We've ordered several of these to the office a...,4
2,Ordered delivery from here last night with my ...,4
3,Ordered some takeout the other night and had n...,3
4,We arrived in town and got to see our son. Dur...,4
5,The Pie was delicious!! The pie doesn't look ...,3
6,Review is for the Melissa King (Top Chef fame)...,4
7,Was excited to try the Souvla x SPG pizza and ...,3
8,The food here is good and creative. They have ...,4
9,Pizza was tasty. Pretty simple to get it deliv...,4


In [ ]:
#Let's inspect a review
df.review.iloc[5]

"The Pie was delicious!! \xa0The pie doesn't look as big in the pictures but this pizza has girth. I thought it wouldn't be enough to share one pie with two people but I was wrong. Fluffy dough and the edges were definitely the catch. \xa0It reminded me of a fresh focaccia bread. \xa0I'd recommend getting a side dish with one pie for two people at most.Super friendly owner that checked in with almost every customer in the restaurant. \xa0Check this place out if you're looking for an adventurous pizza."

In [ ]:
df.sentiment.mean()

3.5454545454545454